<a href="https://colab.research.google.com/github/alba-linares/ecoinfo_densidad_pinar/blob/main/TF_ECOINFO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install eefolium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 1.2 MB/s eta 0:00:00


In [ ]:
import ee

In [ ]:
import eefolium as geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-albalinares20')

In [ ]:
zone = ee.FeatureCollection('projects/ee-albalinares20/assets/grupo_1_25830')

In [ ]:
def get_LSWI(img):
  return img.addBands(img.normalizedDifference(["B4","B5"]).rename("LSWI")) #Son las bandas B4 = NIR y B5 = SWIR-1, porque LSWI = (N - S1)/(N + S1)

In [ ]:
l7collectionLSWI = (
    ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") #SR = surface reflectLSWI
    .filterBounds(zone) #extensión, lo que encuadra a la capa
    .filterDate("1999-01-01","2021-12-31")
) #basicamente, son todas las imagenes

l7lswi = l7collection.map(get_LSWI) #.map es un loop. Para cada objeto de la lista, le aplica iterativamente get_LSWI
composite = l7lswi.qualityMosaic("LSWI").clip(zone) #clip es lo que lo encuadra dentro de la zona
#En base a la banda que hemos nombrado LSWI
#.qualityMosaic te escoge el valor mas alto en base a LSWI
#Option B:
#Mean values as an alternative (yearly mean of each pixel): #Normalmente se usa la mediana, mejor que la media, porque es el más frecuente, y la media está muy afectada por outliers
#composite ...
band = composite.select("LSWI")

#Carga de la colección de imágenes Landsat 7 con reflectancia superficial filtrados por fecha y zona
l7collection = (
    ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") #SR = surface reflectance de Landsat 7
    .filterBounds(zone) #extensión, lo que encuadra a la capa
    .filterDate("1999-01-01","2021-12-31") #periodo en el que ha estado Landsat 7 recopilando imágenes
)

#CÁLCULO DEL ÍNDICE LSWI (Land Surface Water Index) PARA CADA IMAGEN EN LA COLECCIÓN, crea un mosaico de calidad basado en el LSWI y lo recorta según la zona definida. Posteriormente, calcula el promedio del LSWI para cada año en el rango de años especificado, almacenando las bandas resultantes en una lista llamada bands.

l7lswi = l7collection.map(get_LSWI) #.map es un loop. Para cada objeto de la lista, le aplica iterativamente la función get_LSWI
#clip es lo que lo encuadra dentro de la zona del humedal
#En base a la banda que hemos nombrado LSWI
#.Reducer.mean te escoge el valor medio en base a LSWI
composite = l7lswi.select('LSWI').reduce(ee.Reducer.mean()).rename('LSWI').clip(zone)
band = composite.select("LSWI")

#PARA CALCULAR EL LSWI EN VARIOS AÑOS:
bands = []

years = range(1999,2022) #Porque empieza a contar en 1999 y termina en 2021
for year in years:
  str_year=str(year)
  print(str_year)
  l7collection = (
    ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
    .filterBounds(zone)
    .filterDate(str_year + "-01-01",str_year + "-12-31")
)

  l7lswi = l7collection.map(get_LSWI)
  composite = l7lswi.select('LSWI').reduce(ee.Reducer.mean()).rename('LSWI').clip(zone)

#SE ALMACENAN LAS BANDAS DE LAS IMÁGENES CON EL CÁLCULO DEL LSWI EN bands
  band = composite.select("LSWI")
  bands.append(band) #band tiene 23 elementos, uno para cada año

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [ ]:
m = geemap.Map() #inicializamos un mapa vacio
m.centerObject(zone, zoom=10)
vis_params = {"min": 0, "max": 1, "palette":['#D01C8B','#F1B6DA','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529']} #nos interesan sobre todo entre 0.3 #'FFFFFF','CE7E45','D7923D','F1B555','FCD163','011301'
#['D01C8B','F1B6DA','F7F7F7','B8E186','276419']
m.addLayer(band,vis_params,"LSWI")

m

In [ ]:
bands = []

years = range(1999,2022) #porque empieza a contar en 2011 y termina en 2020
for year in years:
  str_year=str(year)
  print(str_year)
  l7collection = (
    ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
    .filterBounds(zone)
    .filterDate(str_year + "-01-01",str_year + "-12-31") #concatenar
)

  l7lswi = l7collection.map(get_LSWI) #.map es un loop. Para cada objeto de la lista, le aplica iterativamente get_LSWI
  composite = l7lswi.qualityMosaic("LSWI").clip(zone) #clip es lo que lo encuadra dentro de la zona
#En base a la banda que hemos nombrado LSWI
#.qualityMosaic te escoge el valor mas alto en base a LSWI
  band = composite.select("LSWI")
  bands.append(band) #band tiene 10 elementos, uno para cada año

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [ ]:
m = geemap.Map() #inicializamos un mapa vacio
m.centerObject(zone, zoom=10)
vis_params = {"min": 0, "max": 1, "palette":['#D01C8B', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718','74A901', '66A000', '529400', '3E8601', '207401', '056201','004C00', '023B01', '012E01', '011D01', '011301']} #nos interesan sobre todo entre 0.3 #'FFFFFF','CE7E45','D7923D','F1B555','FCD163','011301'
#['D01C8B','F1B6DA','F7F7F7','B8E186','276419']
#['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718','74A901', '66A000', '529400', '3E8601', '207401', '056201','004C00', '023B01', '012E01', '011D01', '011301'] la del profe
#['#D01C8B','#F1B6DA','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529'] moradito a verde
for posicion, banda in enumerate(bands):
  m.addLayer(banda,vis_params,"LSWI_"+str(1999 + posicion))
m.addLayerControl() #Para escoger que mapas ver con un desplegable

m




KeyboardInterrupt: 

**LSWI value in a specific point (lat,long)**

In [ ]:
lat = 37.0311
lon = -3.3259
point = ee.Geometry.Point([lon,lat])

In [ ]:
data_point = []
for band in bands:
  data_point.append(band.reduceRegion(ee.Reducer.first(), point, 30).get("LSWI").getInfo()) #donde point puede ser también poligono o linea, hay que definir una geometria
  #30 es la escala o tamaño del punto, que hemos puesto para que coincida con el tamaño de pixel para que cuadre, pero puede ser otro
data_point #el loop nos da 10 datos que corresponde con el valor de LSWI en los 10 años

[0.2708846926689148,
 0.35672202706336975,
 0.29898107051849365,
 0.3055393695831299,
 0.31735706329345703,
 0.40142637491226196,
 0.35356849431991577,
 0.3559139668941498,
 0.3727618157863617,
 0.40182793140411377]

First, connect to your Google Drive to be able to export the csv file #O sea, guardar en csv los puntos de LSWI. Guay si necesito exportar muchos datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.DataFrame({"LSWI": data_point}, index=range(2011,2021))
df.index = df.index.rename("Year") #para saber que cada fila corresponde a cada año

#Create a csv file with the results and export it
df.to_csv('/content/drive/My Drive/lswi.csv')

df.plot() #grafico para ver los datos

What about the mean value of Sierra Nevada? (Protected area means)

In [ ]:
data_sn = []
for band in bands:
  data_sn.append(band.reduceRegion(ee.Reducer.mean(), zone, 30).get("LSWI").getInfo()) #zone es el poligono shapefile del Parque Nacional de Sierra Nevada
data_sn

[0.504895104152331,
 0.6423854597323088,
 0.6854132815923729,
 0.7150197219227723,
 0.7221019395575818,
 0.5265208473117863,
 0.7762805811436287,
 0.3910329956296074,
 0.30623718289915886,
 0.5224290573417334,
 0.7500629270519826,
 0.3064322964128012,
 0.5310703871928856,
 0.5193893600977082,
 0.5065872057293718,
 0.5535088368420267,
 0.5553271116613879,
 0.5869226904232507,
 0.7738536872723485,
 0.698035135257409,
 0.4360516911892327,
 0.6325524984666578,
 0.6424137732094783]

In [ ]:
df = pd.DataFrame({"LSWI": data_sn}, index = range(2011,2021))
df.index = df.index.rename("Year")

df.plot()

Now we want to export the LSWI time series rasters to our Google Drive folder:

In [ ]:
for i, band in enumerate(bands):
  geemap.ee_export_image_to_drive(
      band, "lswi_"+str(2011+i), folder="GEE_ECOINF", region=zone.geometry(),scale=30
  ) #nos da 10 mapas raster .tif, uno para cada año
  #.geometry() y no zone solo, region= te pide una propiedad de geometría de un objeto

We will use the Mann-Kendall trend test to detect a trend in a series of values. Mann-Kendall trend test is a nonparametric test

In [ ]:
#Generate a single file with the different layers (bands)
stack_collection = ee.ImageCollection(bands)
#Hasta ahora, teniamos una lista de objetos de python, y cada imagen, que era un año, con sus bandas, la metiamos en una lista de python.
#Esto no es el caso con ImageCollection, que es un objeto de GoogleEarthEngine
lswis = stack_collection.toBands()

In [ ]:
lswi_trend = lswis.reduce(ee.Reducer.kendallsCorrelation())
tau = lswi_trend.select("tau") # Negative values indicate a decreasing trend and positive values indicate a increasing trend

In [ ]:
m = geemap.Map()
m.centerObject(zone, zoom=10)
vis_params = {"min": -1, "max": 1, "palette": ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']}
#he añadido un color para que sea (dos negativos, uno neutral, dos positivos). Antes era red, orange, white, green
#['#d73027','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850']
# ["red","orange", "white","greenyellow","darkgreen"]
#7 clases  ['#d73027','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850']
m.addLayer(tau, vis_params, "Mann-Kendal")
m

In [ ]:
geemap.ee_export_image_to_drive(
    tau, "tau", folder="GEE_ECOINF",region=zone.geometry(), scale=30
)

In [ ]:
geemap.ee_export_image_to_drive(
    band, "LSWI", folder="GEE_ECOINF",region=zone.geometry(), scale=30
)

Exporting LSWI ...
